In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [9]:
name = 'gbm0'

In [12]:
submission = pd.read_csv('../Data/sample_submission.csv')

In [2]:
train_X = pd.read_csv('../Data/train_X_xgb.csv',index_col=0)
train_y = pd.read_csv('../Data/train_y_xgb.csv',index_col=0)
val_X = pd.read_csv('../Data/val_X_xgb.csv',index_col=0)
val_y = pd.read_csv('../Data/val_y_xgb.csv',index_col=0)
test_X = pd.read_csv('../Data/test_X_xgb.csv',index_col=0)

In [3]:
dtrain = xgb.DMatrix(train_X.values, label=train_y.values)
dval = xgb.DMatrix(val_X.values, label=val_y.values)
dtest = xgb.DMatrix(test_X.values)

In [4]:
# set margin
mean = train_y.values.mean()
dtrain.set_base_margin(np.ones(train_X.shape[0])*mean)
dval.set_base_margin(np.ones(val_X.shape[0])*mean)
dtest.set_base_margin(np.ones(test_X.shape[0])*mean)

Build model

In [5]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [6]:
param = {}
param['eta'] = 0.01
param['gamma '] = 1
param['max_depth'] = 6
param['nthread'] = 4
param['colsample_bylevel'] = 0.5
param['subsample'] = 0.5
param['eval_metric'] = 'mae'
param['tree_method'] = 'hist'
num_round = 1000 

In [7]:
def MAE_obj(preds, dtrain):
    labels = dtrain.get_label()
    grad = np.where(preds>labels,1.0,-1.0)
    hess = np.ones_like(grad)
    return grad, hess

In [8]:
GBM = xgb.train(param, dtrain, num_round, watchlist, obj=MAE_obj,verbose_eval=100)

[0]	train-mae:2.99679	val-mae:3.2041
[100]	train-mae:2.54604	val-mae:2.86332
[200]	train-mae:2.20655	val-mae:2.62128
[300]	train-mae:1.9639	val-mae:2.45168
[400]	train-mae:1.79788	val-mae:2.35825
[500]	train-mae:1.68132	val-mae:2.32287
[600]	train-mae:1.58882	val-mae:2.30614
[700]	train-mae:1.5084	val-mae:2.30226
[800]	train-mae:1.43748	val-mae:2.30396
[900]	train-mae:1.37589	val-mae:2.30909
[999]	train-mae:1.31725	val-mae:2.31955


In [10]:
GBM.save_model('../Model/'+name+'.model')

In [13]:
submission.iloc[:,1] = GBM.predict(dtest,ntree_limit=GBM.best_ntree_limit)
submission.to_csv('../Submission/'+name+'.csv',index=False)